In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sqlite3

%config InlineBackend.figure_formats = ['svg']
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Load Data

In [79]:
# Specify categorical columns
df = pd.read_csv('../temperament.csv', na_values=' ',
                 dtype={'sex': 'category', 're_race': 'category', 'lunch': 'category'})
df = df.set_index('id')
df.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
11005,2,0,1,1,61.0,11.0,2.0,19.0,2007.0,2.0,27.0,2007.0,7.824778,0,2,1,0,1,4/24/1999,24,4,1999,9.0,7.0,2/19/2007,1,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.761905,1.909091,1.6,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.116485,-0.2774,-0.4625,-0.053054,-1.11316,0.01796,-0.107848,0.001740,0,-0.033515,-0.054985,0.071977,-0.016677,0.286778,0.120315,-0.445495,0.033515,0.045385,-0.071977,-0.104285,0.016677,-0.009404,-0.009308,0.020195,0.019718,-0.004679,0.0,0.009404,0.012734,-0.020195,-0.029260,0.004679,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11008,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.427789,0,2,2,1,0,9/2/1999,2,9,1999,5.0,7.0,2/5/2007,1,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.285714,1.181818,1.4,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.132918,-0.2774,0.5375,0.051265,-0.11316,1.01796,-0.012823,0.115353,0,-0.039809,-0.062327,0.062536,-0.002691,-0.179889,-0.264300,-0.921686,0.039809,0.052727,-0.062536,-0.120718,0.002691,0.007408,0.018535,-0.011637,-0.039528,0.000501,0.0,-0.007408,-0.009812,0.011637,0.022464,-0.000501,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11009,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,8.295688,0,2,2,1,1,10/20/1998,20,10,1998,3.0,8.0,2/5/2007,1,3.0,3.0,3.0,1.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,4.0,3.0,3.0,1.0,1.0,1.0,2.047619,2.090909,2.0,...,NaN,NaN,NaN,1.000000,4.666667,-1,0.004621,-0.2774,-0.3375,0.006330,-0.98816,0.14296,0.014802,-0.002141,0,-0.064393,0.120259,-0.030677,-0.009093,0.631222,-0.033531,-0.159781,0.064393,-0.129859,0.030677,0.007579,0.009093,-0.040247,0.061780,-0.019174,0.002374,-0.005683,0.0,0.040247,-0.081165,0.019174,0.004737,0.005683,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11011,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.696099,0,2,2,1,0,5/27/1999,27,5,1999,8.0,7.0,2/5/2007,1,2.0,3.0,4.0,2.0,4.0,3.0,1.0,1.0,4.0,3.0,4.0,5.0,1.0,3.0,2.0,2.0,4.0,2.0,3.0,4.0,1.0,2.761905,2.727273,2.8,...,NaN,NaN,NaN,1.000000,3.000000,-1,0.009721,-0.2774,-0.7125,0.007880,-1.36316,-0.23204,0.013467,0.002292,0,-0.045452,0.034538,-0.049376,0.078814,0.620111,0.043392,0.554505,0.045452,-0.044138,0.049376,0.002479,-0.078814,-0.027903,0.01

# Select rows for Cohort 1 with no missing data

In [80]:
aggdf = df[(df.cohort == 1) & (~df.mt1tovertagg.isnull()) & (~df.mt2tovertagg.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1picontrol.isnull())]
aggdf.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
1002,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.039014,0,2,2,1,0,12/28/1998,28,12,1998,0.0,7.0,1/11/2006,1,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,5.0,5.0,5.0,3.571429,4.818182,2.2,...,-1.135892,1.852989,0.635096,1.000000,1.000000,-1,-0.088186,-0.2774,-0.0875,0.018604,-0.73816,0.39296,0.079563,-0.042355,1,-0.041757,-0.082457,-0.011814,-0.040029,-0.921556,-0.264300,1.364029,0.041757,0.072857,0.011814,0.100386,0.040029,0.038740,0.085678,0.010961,0.091630,0.037137,0.0,-0.038740,-0.067594,-0.010961,-0.093133,-0.037137,2.498208,2.498208,-0.220192,0.0,0.0,0.0,-2.302237,0.0,2.909522,-0.393039,2.883833,-0.385625
1004,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.468857,0,2,6,0,0,7/24/1998,24,7,1998,5.0,7.0,1/11/2006,1,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.476190,1.909091,1.0,...,-1.619677,0.999122,0.910435,1.000000,2.666667,-1,0.089335,-0.2774,0.0375,-0.003319,-0.61316,0.51796,-0.042759,0.036120,1,-0.016547,-0.086659,-0.012865,0.009341,-0.274333,-0.264300,-0.731210,0.016547,0.077059,0.012865,-0.077135,-0.009341,0.004642,0.033956,0.003609,-0.035005,-0.002621,0.0,-0.004642,-0.021618,-0.003609,0.021639,0.002621,-0.762949,-0.762949,-0.029049,0.0,0.0,0.0,0.209302,0.0,-0.381589,-0.393039,-0.377324,-0.385625
1006,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.690623,0,2,2,1,1,5/4/1998,4,5,1998,8.0,7.0,1/11/2006,1,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.523810,4.727273,4.3,...,2.452864,-0.439749,-0.332146,1.000000,1.000000,-1,0.021370,-0.2774,-0.4625,-0.000969,-1.11316,0.01796,-0.001971,0.000032,1,-0.067819,0.067157,-0.060559,0.041535,0.622889,0.120315,2.316410,0.067819,-0.076757,0.060559,-0.009170,-0.041535,-0.041823,0.033940,-0.037346,0.010800,0.025614,0.0,0.041823,-0.047335,0.037346,-0.005655,-0.025614,1.912953,1.912953,-0.893033,0.0,0.0,0.0,1.191557,0.0,2.806675,-0.875793,2.781922,-0.868969
1007,1,0,1,1,17.0,1.0,1.0,9.0,2006.0,1.0,11.0,2006.0,7.403149,0,2,3,0,0,8/15/1998,15,8,1998,4.0,7.0,1/9/2006,1,3.0,1.0,5.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,2.0,1.952381,2.090909,1.8,...,0.739859,0.489876,-0.021501,1.666667,2.000000,-1,-

# Select necessary columns for the analyses

In [82]:
oaggdf = aggdf[['t1age', 're_race', 'lunch', 'sex', 'mt1tovertagg', 'mt2tovertagg',
                'mt1overtvic', 'mt1tovertvic', 'mt1picontrol']]

In [83]:
oaggdf.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
id,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125


In [84]:
oaggdf.dtypes

t1age            float64
re_race         category
lunch           category
sex             category
mt1tovertagg     float64
mt2tovertagg     float64
mt1overtvic      float64
mt1tovertvic     float64
mt1picontrol     float64
dtype: object

In [85]:
oaggdf.shape

(283, 9)

In [86]:
oaggdf.describe()

,t1age,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
count,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
mean,7.945891,1.448763,1.552120,2.246386,1.585680,3.337498
std,0.319879,0.893747,0.946098,0.883924,0.564941,0.585106
min,7.039014,1.000000,1.000000,1.000000,1.000000,1.375000
25%,7.691992,1.000000,1.000000,1.545455,1.181818,3.000000
50%,7.967146,1.000000,1.000000,2.090909,1.454545,3.375000
75%,8.171116,1.500000,1.750000,2.636364,1.818182,3.750000
max,9.117043,5.000000,5.000000,4.818182,4.181818,4.875000
